In [1]:
from route import Route
from route_api import RouteAPI

current_position = {'longitude': 130.875,
                    'latitude': -23.7}
final_position = {'longitude': 133.8750197,
                  'latitude': -23.7090616}
delta_spacing = 2000.0 # in meters
number_sites = 150

route = Route()
api_route = RouteAPI(route)

api_route_df = api_route.cut_route_data(current_position, number_sites=number_sites, print_is_requested=True)
# api_route_df = api_route.cut_route_data(current_position, number_sites=number_sites, delta_spacing=delta_spacing, print_is_requested=True)
# api_route_df = api_route.cut_route_data(current_position=current_position, final_position=final_position, number_sites=number_sites, print_is_requested=False)
# route.find_closest_row(current_position)

# route.insert_to_control_stops()

Nearest index in csv file: 4895
Nearest index in dataframe: 4893
      longitude   latitude  altitudeSmoothed  inclinationSmoothed  \
0    133.238422 -24.544813        425.339407            -0.015766   
1    133.215850 -24.623148        437.698868             0.005529   
2    133.190346 -24.701845        434.383256            -0.001717   
3    133.179177 -24.780813        413.137552            -0.000181   
4    133.194052 -24.863975        420.065884             0.004471   
..          ...        ...               ...                  ...   
145  138.517066 -34.610731         15.367843             0.001466   
146  138.559377 -34.686748         12.585737            -0.000915   
147  138.593632 -34.765144         15.010025             0.002416   
148  138.600353 -34.849753         10.631818             0.004716   
149  138.600712 -34.928189         64.449372            -0.011632   

      cumDistance  maxSpeed         theta  timeAtMaxSpeed  cumTimeAtMaxSpeed  \
0    1.620901e+06       13

In [2]:
from Solcast_executer import SolcastExecuter
solcast_executer = SolcastExecuter()

# TODO TIMEOUT ERROR BUG

# solcast_executer.get_forecast(current_position)
raw_df = solcast_executer.get_forecasts(api_route_df, hours_in_advance=48)
print(raw_df)

                                        tt    gh     ff   dd    rh   rr
cumDistance  time                                                      
1.620901e+06 2023-10-14 10:45:00+09:30  24   961  23.76  114  14.5  0.0
             2023-10-14 11:00:00+09:30  25   992  24.12  114  13.9  0.0
             2023-10-14 11:15:00+09:30  25  1018  24.12  115  13.3  0.0
             2023-10-14 11:30:00+09:30  26  1040  24.12  116  12.7  0.0
             2023-10-14 11:45:00+09:30  26  1057  24.12  117  12.2  0.0
...                                     ..   ...    ...  ...   ...  ...
3.026918e+06 2023-10-16 09:45:00+09:30  15   565  23.76  205  49.0  0.0
             2023-10-16 10:00:00+09:30  15   593  23.76  205  48.3  0.0
             2023-10-16 10:15:00+09:30  15   619  23.76  205  47.6  0.0
             2023-10-16 10:30:00+09:30  15   641  23.40  205  46.9  0.0
             2023-10-16 10:45:00+09:30  16   660  23.76  205  46.3  0.0

[28923 rows x 6 columns]


In [3]:
from Meteotest_executer import MeteotestExecuter
from Meteotest_requester import MeteotestRequester
from Meteotest_parser import MeteotestParser

api_parser = MeteotestParser()
api_requester = MeteotestRequester(parser=api_parser, print_is_requested=False)
api_executer = MeteotestExecuter(requester=api_requester)

Response status from siteinfo: OK.


In [4]:
# api_requester.get_site_info(print_is_requested=True)
# api_executer.get_all_site_id

In [5]:
# api_requester.get_site_add(name="0", position=current_position, print_is_requested=True)
# api_executer.add_sites(api_route_df, print_is_requested=True)

In [6]:
# api_requester.get_site_delete(585394, print_is_requested=True)
# api_executer.delete_all_sites()

In [7]:
# site_id = 54689
# name = "404"
# position = {'latitude': 136,
#             'longitude': -30}

# api_requester.get_site_edit(site_id, name=name, position=position)

# api_route_df.index=api_executer.get_all_site_id ########TODO ADAPT DIRECTLY 
# api_executer.edit_sites(api_route_df)

In [8]:
# raw_df = api_requester.get_solar_forecast()
# print(raw_df.head())

# raw_df = api_requester.get_solar_forecast_cloudmove()
# print(raw_df)
# raw_SF_df, _, raw_CM_df, _ = api_executer.get_new_forecasts()

# print(raw_SF_df.head(), raw_CM_df.head())

In [9]:
sites_df = api_requester.get_current_sites

from preprocessor import Preprocessor
preprocessor = Preprocessor()
hours_in_advance = None
preprocessed_df, preprocessed_df = preprocessor.preprocess(api_route_df, sites_df, raw_df, hours_in_advance)
# preprocessor.save_data2folder()
print(preprocessed_df)

                                        temperature  globalIrradiance  \
cumDistance  time                                                       
1.620901e+06 2023-10-14 10:45:00+09:30           27               961   
             2023-10-14 11:00:00+09:30           28               992   
             2023-10-14 11:15:00+09:30           28              1018   
             2023-10-14 11:30:00+09:30           29              1040   
             2023-10-14 11:45:00+09:30           29              1057   
...                                             ...               ...   
3.026918e+06 2023-10-16 09:45:00+09:30           15               565   
             2023-10-16 10:00:00+09:30           18               593   
             2023-10-16 10:15:00+09:30           18               619   
             2023-10-16 10:30:00+09:30           18               641   
             2023-10-16 10:45:00+09:30           19               660   

                                        windSpeed 

In [10]:
from db_query import DbQuerier

db_querier = DbQuerier()

v = db_querier.query_velocity()
soc = db_querier.query_soc()
print(v, soc)

db_querier.save_data2folder()

Data read from C:/Users/giaco/Git_Repositories/aCentauri/can-msg-api/db/.env.
                       time  velocity
0 2023-10-12 14:02:13+09:30       0.0
1 2023-10-12 14:02:13+09:30       0.0
2 2023-10-12 14:02:13+09:30       0.0
3 2023-10-12 14:02:13+09:30       0.0
4 2023-10-12 14:02:13+09:30       0.0                        time      SoC
0 2023-10-12 14:02:57+09:30  0.14008
1 2023-10-12 14:02:57+09:30  0.14008
2 2023-10-12 14:02:56+09:30  0.14008
3 2023-10-12 14:02:55+09:30  0.14008
4 2023-10-12 14:02:54+09:30  0.14008
A folder does not exist!


In [11]:
# import folium

# # Assuming you have these dataframes already
# control_stops_df = route.get_control_stops_data
# sites_df = api_requester.get_current_sites
# rt = route.get_route_data

# # Define the bounds
# min_lng = rt['longitude'].min()
# min_lat = rt['latitude'].min()
# max_lng = rt['longitude'].max()
# max_lat = rt['latitude'].max()
# bounds = [[max_lat, min_lng], [min_lat, max_lng]]

# # Create a base map centered around Australia
# middle_lat = (max_lat - min_lat) / 2
# middle_lng = (max_lng - min_lng) / 2
# map = folium.Map(location=[middle_lat, middle_lng],
#                  zoom_start=0)

# # Set the map boundaries to show only Australia
# map.fit_bounds(bounds)

# # Add the route lines
# folium.PolyLine(
#     locations=rt[['latitude', 'longitude']].values.tolist(),
#     color="green",
#     weight=4,
#     opacity=1
# ).add_to(map)

# # Add api sites
# for _, row in sites_df.iterrows():
#     folium.CircleMarker(
#         location=[row['latitude'], row['longitude']],
#         radius=2,
#         color="blue",
#         fill=True,
#         fill_color="blue",
#         fill_opacity=1,
#         tooltip=row['name']
#     ).add_to(map)

# # Add control stops
# for _, row in control_stops_df.iterrows():
#     folium.CircleMarker(
#         location=[row['latitude'], row['longitude']],
#         radius=3,
#         color="orange",
#         fill=True,
#         fill_color="orange",
#         fill_opacity=1,
#         tooltip=row['town'] + ': ' + row['location']
#     ).add_to(map)

# folium.Marker(
#                 location=[current_position['latitude'], current_position['longitude']],
#                 icon=folium.Icon(icon="car", prefix="fa")
#         ).add_to(map)

# map.options['maxBounds'] = bounds
# map.options['maxBoundsViscosity'] = 0.9
# map.options['minZoom'] = 5

# map


In [12]:
# from plotter import Plotter

# plotter = Plotter(route)
# plotter.add_api_sites(api_requester.get_current_sites)
# plotter.add_control_stops(route.get_control_stops_data)
# # plotter.add_current_position(current_position)
# current_position = {'longitude': 132.213897,
#                     'latitude': -14.425286}
# plotter.update_max_speed_distance(current_position)
# plotter.plot

# #TODO 
# #SE SI CHIAMA CLOUDMOVE NON ELIMINARE ALTRI DATI METEO MA UNIRLI
# #SE NON CI SONO DATI PLOTTARE NO DATA

In [22]:
#TODO i dati vengono sovrascritti e non aggiornati nei plot
# tenere ultimo SF e aggiornare dati di CM dove possibile
# se non ci sono dati di SF usare CM e viceversa

# se codice runna per tanto, i dropdown del tempo devono essere aggiornati

# per i weather plot bisogna correggere il cumDistance cosi che da 0 fino al primo cumDistance i valori siano 0 per tutti i tempi
# lo stesso vale dall'ultimo punto fino a route_data['cumDistance'].max()

# cambiare i font

import pandas as pd
import dash
import constants
from dash import Dash, html, dcc
from dash.dependencies import Input, Output, State
# import plotly.graph_objects as go

control_stops_df = route.get_control_stops_data
sites_df = api_requester.get_current_sites

app = Dash(__name__)

# aCe images
url_image = 'https://avatars.githubusercontent.com/u/115261983?s=280&v=4'

## Time
# Slider
height_slider = 1000
unique_times = preprocessed_df.index.get_level_values('time').unique()
slider_marks = {i: time.strftime('%H:%M') for i, time in enumerate(unique_times)}

# Dropdown
unique_days = unique_times.map(lambda x: x.strftime('%d %B %Y')).unique().to_list()

# Update time
update_sec = 50

## Map
route_data = route.get_route_data
min_lng = route_data['longitude'].min()
min_lat = route_data['latitude'].min()
max_lng = route_data['longitude'].max()
max_lat = route_data['latitude'].max()
middle_lat = (max_lat + min_lat) / 2
middle_lng = (max_lng + min_lng) / 2

## Plot
x_cumDistance_km = route_data['cumDistance'] / 1000
x_cumDistance_str = 'Cumulative Distance (km)'

grid_color = 'lightgray'
box_color = 'black'

height_plot = 280

# Control stops, start, and finish lines
control_stops_df = route.get_control_stops_data

control_stops = control_stops_df.iloc[1:-1]
cs_vertical_lines = []
for _, row in control_stops.iterrows():
    x_value = row['cumDistance'] / 1000
    cs_vertical_lines.append(
        dict(type='line', x0=x_value, x1=x_value, y0=0, y1=1, yref='paper', line=dict(color='orange', dash='dash'))
    )

start_finish = control_stops_df.iloc[[0, -1]]
start_finish_lines = []
for _, row in start_finish.iterrows():
    x_value = row['cumDistance'] / 1000
    start_finish_lines.append(
        dict(type='line', x0=x_value, x1=x_value, y0=0, y1=1, yref='paper', line=dict(color='black', width=0.5))
    )
    
vertical_lines = cs_vertical_lines + start_finish_lines


app.layout = html.Div([
    # Header with image and title
    html.Div([
        # Title
        html.H1("Strategy Interface",
                style={'textAlign': 'left', 'margin': '0 auto', 'flex': '1'}),

        # Dropdown
        dcc.Dropdown(options=[{'label': day, 'value': day} for day in unique_days],
                     value=unique_days[0],
                     id='day-month-year-dropdown',
                     style={'width': '200px', 'margin-right': '20px'}),
        
        # Last update
        dcc.Interval(id='interval-update', interval=update_sec*1000, n_intervals=0),
        html.Div([
            html.Div(id='last-update-sf',
                     style={'color': 'gray'}),
            html.Div(id='last-update-cm',
                     style={'color': 'gray'}),
            html.Div(id='last-update-sc',
                     style={'color': 'gray'}),
        ], style={'display': 'flex', 'flexDirection': 'column', 'margin-right': '20px'}),

        # Image
        html.Img(src=url_image,
                 style={'height': '100px', 'width': 'auto', 'margin-left': '20px'}),
        
    ], style={'display': 'flex', 'alignItems': 'center', 'padding-left': '20px', 'padding-right': '20px'}),


    # Container for the columns
    html.Div([
        # Map and static plots
        html.Div(children=[
            # Map
            dcc.Graph(
                id='map',
                config={'displayModeBar': True, 'scrollZoom': True},
                figure={
                    'data': [{
                        'type': 'scattermapbox',
                        'lat': route_data['latitude'],
                        'lon': route_data['longitude'],
                        'mode': 'lines+markers',
                        'marker': {'size': 3},
                        'line': {'color': 'blue'},
                        'hoverinfo': 'none',
                        'name': 'Route'
                    },
                    {
                        'type': 'scattermapbox',
                        'lat': sites_df['latitude'],
                        'lon': sites_df['longitude'],
                        'mode': 'markers',
                        'marker': {'size': 3},
                        'line': {'color': 'white'},
                        'hoverinfo': 'none',
                        'name': 'Weather locations'
                    },
                    {
                        'type': 'scattermapbox',
                        'lat': control_stops_df['latitude'],
                        'lon': control_stops_df['longitude'],
                        'mode': 'markers',
                        'marker': {'size': 5},
                        'line': {'color': 'orange'},
                        'hoverinfo': control_stops_df['town'] + ': ' + control_stops_df['location'],
                        'name': 'Control Stops'
                    },
                    {
                        'type': 'scattermapbox',
                        'lat': [current_position['latitude']],
                        'lon': [current_position['longitude']],
                        'mode': 'markers',
                        'marker': {'size': 10, 'color': 'red'},
                        'hoverinfo': 'none'
                    }],
                    'layout': {
                        'mapbox': {
                            'style': "mapbox://styles/mapbox/satellite-streets-v11",
                            'accesstoken': constants.TOKEN_MAXBOX,
                            'center': {'lat': middle_lat, 'lon': middle_lng},
                            'zoom': 3.5,
                            'bearing': 0,
                            'pitch': 0,
                            'maxBounds': {
                                'ne': {'lat': -10, 'lon': 150},  # Northeast corner
                                'sw': {'lat': -40, 'lon': 110}   # Southwest corner
                            },
                            'uirevision': 'no_change'
                        },
                        'margin': {'l': 0, 'r': 0, 'b': 0, 't': 0},
                        'legend': {
                            'x': 0,
                            'y': 1,
                            'xanchor': 'left',
                            'yanchor': 'top'
                        }
                    }
                }
            ),

            # Max speed
            html.Br(),
            dcc.Graph(
                id='maxSpeed',
                figure={
                    'data': [
                        {
                            'x': x_cumDistance_km,
                            'y': route_data['maxSpeed'],
                            'type': 'scatter',
                            'mode': 'lines'
                        },
                    ],
                    'layout': {
                        'margin': {
                            "r": 0,
                            "t": 0
                            },
                        'xaxis': {
                            'title': x_cumDistance_str,
                            'showgrid': True,
                            'gridcolor': grid_color,
                            'showline': True,
                            'linecolor': box_color
                        },
                        'yaxis': {
                            'title': 'Max Speed (km/h)',
                            'showgrid': True,
                            'gridcolor': grid_color,
                            'showline': True,
                            'linecolor': box_color
                        },
                        'shapes': vertical_lines,
                        'height': height_plot
                    }
                }
            ),

            # Altitude
            html.Br(),
            dcc.Graph(
                id='altitude',
                figure={
                    'data': [
                        {
                            'x': x_cumDistance_km,
                            'y': route_data['altitudeSmoothed'],
                            'type': 'scatter',
                            'mode': 'lines'
                        },
                    ],
                    'layout': {
                        'margin': {
                            "r": 0,
                            "t": 0
                            },
                        'xaxis': {
                            'title': x_cumDistance_str,
                            'showgrid': True,
                            'gridcolor': grid_color,
                            'showline': True,
                            'linecolor': box_color
                        },
                        'yaxis': {
                            'title': 'Altitude (m)',
                            'showgrid': True,
                            'gridcolor': grid_color,
                            'showline': True,
                            'linecolor': box_color
                        },
                        'shapes': vertical_lines,
                        'height': height_plot
                    }
                }
            ),

            # Inclination TODO CONTROLLARE CHE SIA IN GRADI E NON RADIANTI
            html.Br(),
            dcc.Graph(
                id='inclination',
                figure={
                    'data': [
                        {
                            'x': x_cumDistance_km,
                            'y': route_data['inclinationSmoothed'],
                            'type': 'scatter',
                            'mode': 'lines'
                        },
                    ],
                    'layout': {
                        'margin': {
                            "r": 0,
                            "t": 0
                            },
                        'xaxis': {
                            'title': x_cumDistance_str,
                            'showgrid': True,
                            'gridcolor': grid_color,
                            'showline': True,
                            'linecolor': box_color
                        },
                        'yaxis': {
                            'title': 'Inclination (°)',
                            'showgrid': True,
                            'gridcolor': grid_color,
                            'showline': True,
                            'linecolor': box_color
                        },
                        'shapes': vertical_lines,
                        'height': height_plot
                    }
                }
            )
        ], style={'padding': 10, 'flex': 1}),

        # Time slider
        html.Div(
            dcc.RangeSlider(
                id='hour-minute-range-slider',
                min=0,
                max=len(unique_days) - 1,
                value=[0, len(unique_days) - 1],
                marks=slider_marks,
                vertical=True,
                step=1,
                verticalHeight=height_slider
            ),
            style={'width': '70px', 'height':height_slider},
            id='hour-minute-range-slider-container'
        ),

        # Weather plots
        html.Div(children=[
            dcc.Graph(id='plot-globalIrradiance'),
            dcc.Graph(id='plot-temperature'),
            dcc.Graph(id='plot-wind'),
            dcc.Graph(id='plot-precipitation'),

        ], style={'padding': 10, 'flex': 1})
    ], style={'display': 'flex', 'flex-direction': 'row'}),
    
], style={'display': 'flex', 'flex-direction': 'column'})

## Callbacks
# Last update
@app.callback(
    Output('last-update-sf', 'children'),
    Output('last-update-cm', 'children'),
    Output('last-update-sc', 'children'),
    Input('interval-update', 'n_intervals')
)
def update_output_div(n):
    time_SF_str = api_requester.get_solar_forecast_last_time
    if time_SF_str is pd.NaT:
        print_SF = "Last update SF: No data available"
    else:
        print_SF = f"Last update SF: {time_SF_str.strftime('%d %B %Y at %H:%M')}"

    time_CM_str = api_requester.get_solar_forecast_cloudmove_last_time
    if time_CM_str is pd.NaT:
        print_CM = "Last update CM: No data available"
    else:
        print_CM = f"Last update CM: {time_CM_str.strftime('%d %B %Y at %H:%M')}"

    time_SC_str = solcast_executer.get_solcast_last_time
    if time_SC_str is pd.NaT:
        print_SC = "Last update SC: No data available"
    else:
        print_SC = f"Last update SC: {time_SC_str.strftime('%d %B %Y at %H:%M')}"

    return print_SF, print_CM, print_SC


# Time callback
@app.callback(
    [Output('hour-minute-range-slider', 'max'),
     Output('hour-minute-range-slider', 'marks'),
     Output('hour-minute-range-slider', 'value'),
     Output('hour-minute-range-slider-container', 'style')],
     Input('day-month-year-dropdown', 'value')
)
def update_slider(day_month_year):
    min_value = 0
    now = pd.Timestamp.now(tz=constants.TIMEZONE)
    
    unique_times_days = unique_times[unique_times.map(lambda x: x.strftime('%d %B %Y')) == day_month_year]
    slider_marks = {i: {'label': time.strftime('%H:%M')} for i, time in enumerate(unique_times_days)}
    
    next_time_index = unique_times_days.searchsorted(now)
    if next_time_index < len(unique_times_days) and now.day == pd.to_datetime(day_month_year).day:
        slider_marks[next_time_index]['style'] = {'color': 'red'}
        min_value = next_time_index
    
    max_val = len(unique_times_days) - 1
    value = [min_value, max_val]
    style = {'width': '100px', 'height': f'{max_val*60}px'}
    
    return max_val, slider_marks, value, style

# Axis callback
@app.callback(
    [Output('maxSpeed', 'figure'),
     Output('altitude', 'figure'),
     Output('inclination', 'figure'),
     Output('plot-globalIrradiance', 'figure', allow_duplicate=True),
     Output('plot-temperature', 'figure', allow_duplicate=True),
     Output('plot-wind', 'figure', allow_duplicate=True),
     Output('plot-precipitation', 'figure', allow_duplicate=True)],
    [Input('maxSpeed', 'relayoutData'),
     Input('altitude', 'relayoutData'),
     Input('inclination', 'relayoutData'),
     Input('plot-globalIrradiance', 'relayoutData'),
     Input('plot-temperature', 'relayoutData'),
     Input('plot-wind', 'relayoutData'),
     Input('plot-precipitation', 'relayoutData'),],
    [State('maxSpeed', 'figure'),
     State('altitude', 'figure'),
     State('inclination', 'figure'),
     State('plot-globalIrradiance', 'figure'),
     State('plot-temperature', 'figure'),
     State('plot-wind', 'figure'),
     State('plot-precipitation', 'figure')],
    prevent_initial_call=True)
def update_xaxis(v_max, h, dh, gh, T, w, rr, fig_v_max, fig_h, fig_dh, fig_gh, fig_T, fig_w, fig_rr):
    # Get the callback context to determine which input triggered the callback
    ctx = dash.callback_context
    if not ctx.triggered:
        raise dash.exceptions.PreventUpdate
    input_id = ctx.triggered[0]['prop_id'].split('.')[0]

    trigger_value = ctx.triggered[0]['value']
    # print(input_id)
    # print(trigger_value)
    # print(trigger_value.keys())
    # print("------------------")

    # Check if autorange is set for any of the graphs
    if 'autosize' in trigger_value and trigger_value['autosize']:
        if trigger_value['autosize']:
            raise dash.exceptions.PreventUpdate
    
    elif 'xaxis.autorange' in trigger_value and trigger_value['xaxis.autorange']:
        # If autorange is set for any graph, let all graphs determine their x-axis range automatically
        fig_v_max['layout']['xaxis']['autorange'] = True
        fig_h['layout']['xaxis']['autorange'] = True
        fig_dh['layout']['xaxis']['autorange'] = True
        fig_gh['layout']['xaxis']['autorange'] = True
        fig_T['layout']['xaxis']['autorange'] = True
        fig_w['layout']['xaxis']['autorange'] = True
        fig_rr['layout']['xaxis']['autorange'] = True

    else:
        # Determine the new x-axis range
        if input_id == 'maxSpeed':
            xaxis_range = [v_max['xaxis.range[0]'], v_max['xaxis.range[1]']]
        elif input_id == 'altitude':
            xaxis_range = [h['xaxis.range[0]'], h['xaxis.range[1]']]
        elif input_id == 'inclination':
            xaxis_range = [dh['xaxis.range[0]'], dh['xaxis.range[1]']]
        elif input_id == 'plot-globalIrradiance':
            xaxis_range = [gh['xaxis.range[0]'], gh['xaxis.range[1]']]
        elif input_id == 'plot-temperature':
            xaxis_range = [T['xaxis.range[0]'], T['xaxis.range[1]']]
        elif input_id == 'plot-wind':
            xaxis_range = [w['xaxis.range[0]'], w['xaxis.range[1]']]
        elif input_id == 'plot-precipitation':
            xaxis_range = [rr['xaxis.range[0]'], rr['xaxis.range[1]']]
        else:
            raise dash.exceptions.PreventUpdate

        # Update the x-axis range for all three figures and set autorange to False
        fig_v_max['layout']['xaxis']['range'] = xaxis_range
        fig_v_max['layout']['xaxis']['autorange'] = False

        fig_h['layout']['xaxis']['range'] = xaxis_range
        fig_h['layout']['xaxis']['autorange'] = False

        fig_dh['layout']['xaxis']['range'] = xaxis_range
        fig_dh['layout']['xaxis']['autorange'] = False

        fig_gh['layout']['xaxis']['range'] = xaxis_range
        fig_gh['layout']['xaxis']['autorange'] = False

        fig_T['layout']['xaxis']['range'] = xaxis_range
        fig_T['layout']['xaxis']['autorange'] = False

        fig_w['layout']['xaxis']['range'] = xaxis_range
        fig_w['layout']['xaxis']['autorange'] = False

        fig_rr['layout']['xaxis']['range'] = xaxis_range
        fig_rr['layout']['xaxis']['autorange'] = False

    return fig_v_max, fig_h, fig_dh, fig_gh, fig_T, fig_w, fig_rr

#####TODO DEBUG ZOOM SOLO VERTICALE: PREVENTUPDATE

####TODO CAMBIO TEMPO NON SI RIPLOTTA TUTTO LA FIGURE, MA SI AGGIORNANO SOLO DATI CON AXIS VECCHIE
# Weather plots
@app.callback(
    [Output('plot-globalIrradiance', 'figure'),
     Output('plot-temperature', 'figure'),
     Output('plot-wind', 'figure'),
     Output('plot-precipitation', 'figure')],
    [Input('day-month-year-dropdown', 'value'),
    Input('hour-minute-range-slider', 'value'),
    Input('hour-minute-range-slider', 'marks')])
def update_graph(day_month_year, hour_minute, slider_marks):
    # Extract timezone from the DataFrame's index
    timezone = preprocessed_df.index.get_level_values('time').tz

    # Create the start and end timestamps for the day
    start_time = pd.Timestamp(day_month_year).tz_localize(timezone)
    end_time = start_time + pd.Timedelta(days=1)

    # Filter the DataFrame for the specific day
    data_for_selected_day = preprocessed_df[(preprocessed_df.index.get_level_values('time') >= start_time) & 
                            (preprocessed_df.index.get_level_values('time') < end_time)]
    
    # Create the start and end timestamps for the range
    first_datetime_str = day_month_year + " " + slider_marks[f'{hour_minute[0]}'] # Slider marks are str and hour_minute int
    first_datetime_obj = pd.Timestamp(first_datetime_str).tz_localize(timezone)
    last_datetime_str = day_month_year + " " + slider_marks[f'{hour_minute[1]}'] # Slider marks are str and hour_minute int
    last_datetime_obj = pd.Timestamp(last_datetime_str).tz_localize(timezone)

    # Create a mask to select the data in the range
    mask = (data_for_selected_day.index.get_level_values('time') >= first_datetime_obj) & (data_for_selected_day.index.get_level_values('time') <= last_datetime_obj)
    data_in_range_df = data_for_selected_day[mask]

    x_cumDistance_range_km = data_in_range_df.index.get_level_values('cumDistance').unique() / 1000

    figure_gh={
        'data': [{
            'x': x_cumDistance_range_km,
            'y': data_in_range_df['globalIrradiance'].groupby(level='cumDistance').mean(),
            'type': 'scatter',
            'mode': 'lines'}],
        'layout': {
            'margin': {"r": 0,
                       "t": 0},
            'xaxis': {
                'title': x_cumDistance_str,
                'showgrid': True,
                'gridcolor': grid_color,
                'showline': True,
                'linecolor': box_color},
            'yaxis': {
                'title': 'Global irradiance (W/m²)',
                'showgrid': True,
                'gridcolor': grid_color,
                'showline': True,
                'linecolor': box_color},
            'shapes': [{
                'type': 'line',
                'x0': x_cumDistance_range_km.min(),
                'x1': x_cumDistance_range_km.max(),
                'y0': data_for_selected_day['globalIrradiance'].max(),
                'y1': data_for_selected_day['globalIrradiance'].max(),
                'line': {
                    'color': 'red',
                    'width': 1}
                },
            ] + vertical_lines,
            'height': height_plot
        }
    }

    figure_tt={
        'data': [{
            'x': x_cumDistance_range_km,
            'y': data_in_range_df['temperature'].groupby(level='cumDistance').mean(),
            'type': 'scatter',
            'mode': 'lines'}],
        'layout': {
            'margin': {"r": 0,
                       "t": 0},
            'xaxis': {
                'title': x_cumDistance_str,
                'showgrid': True,
                'gridcolor': grid_color,
                'showline': True,
                'linecolor': box_color},
            'yaxis': {
                'title': 'Temperature (°C)',
                'showgrid': True,
                'gridcolor': grid_color,
                'showline': True,
                'linecolor': box_color},
            'shapes': [{
                'type': 'line',
                'x0': x_cumDistance_range_km.min(),
                'x1': x_cumDistance_range_km.max(),
                'y0': data_for_selected_day['temperature'].max(),
                'y1': data_for_selected_day['temperature'].max(),
                'line': {
                    'color': 'red',
                    'width': 1}
                },
                {
                'type': 'line',
                'x0': x_cumDistance_range_km.min(),
                'x1': x_cumDistance_range_km.max(),
                'y0': data_for_selected_day['temperature'].min(),
                'y1': data_for_selected_day['temperature'].min(),
                'line': {
                    'color': 'blue',
                    'width': 1}
                }
            ] + vertical_lines,
            'height': height_plot
        }
    }

    figure_ww={
        'data': [{
            # 'x': x_cumDistance_range_km,
            # 'y': data_in_range_df['windGust'].groupby(level='cumDistance').mean() * 3.6,
            # 'type': 'scatter',
            # 'mode': 'lines',
            # 'name': 'Wind gust'},
            # {
            'x': x_cumDistance_range_km,
            'y': data_in_range_df['windSpeed'].groupby(level='cumDistance').mean() * 3.6,
            'type': 'scatter',
            'mode': 'lines',
            'name': 'Wind speed'},
            {
            'x': x_cumDistance_range_km,
            'y': data_in_range_df['sideWind'].groupby(level='cumDistance').mean() * 3.6,
            'type': 'scatter',
            'mode': 'lines',
            'name': 'Side wind'},
            {
            'x': x_cumDistance_range_km,
            'y': data_in_range_df['frontWind'].groupby(level='cumDistance').mean() * 3.6,
            'type': 'scatter',
            'mode': 'lines',
            'name': 'Front wind'},
            ],
        'layout': {
            'margin': {"r": 0,
                       "t": 0},
            'xaxis': {
                'title': x_cumDistance_str,
                'showgrid': True,
                'gridcolor': grid_color,
                'showline': True,
                'linecolor': box_color},
            'yaxis': {
                'title': 'Wind (km/h)',
                'showgrid': True,
                'gridcolor': grid_color,
                'showline': True,
                'linecolor': box_color},
            'height': height_plot,
            'legend': {
                'x': 0.2,
                'y': 1,
                'xanchor': 'left',
                'yanchor': 'top'},
            'shapes': vertical_lines
        }
    }

    figure_rr={
        'data': [{
            'x': x_cumDistance_range_km,
            'y': data_in_range_df['precipitation'].groupby(level='cumDistance').mean(),
            'type': 'scatter',
            'mode': 'lines'},
            ],
        'layout': {
            'margin': {"r": 0,
                       "t": 0},
            'xaxis': {
                'title': x_cumDistance_str,
                'showgrid': True,
                'gridcolor': grid_color,
                'showline': True,
                'linecolor': box_color},
            'yaxis': {
                'title': 'Precipitation (mm)',
                'showgrid': True,
                'gridcolor': grid_color,
                'showline': True,
                'linecolor': box_color},
            'shapes': vertical_lines,
            'height': height_plot,
            'legend': {
                'x': 0.2,
                'y': 1,
                'xanchor': 'left',
                'yanchor': 'top'
            }
        }
    }

    return figure_gh, figure_tt, figure_ww, figure_rr

if __name__ == '__main__':
    app.run_server(debug=True, jupyter_mode="tab")


Dash app running on http://127.0.0.1:8050/


<IPython.core.display.Javascript object>

---------------------------------------------------------------------------
TypeError                                 Traceback (most recent call last)
TypeError: can only concatenate str (not "dict") to str

---------------------------------------------------------------------------
TypeError                                 Traceback (most recent call last)
TypeError: can only concatenate str (not "dict") to str

---------------------------------------------------------------------------
TypeError                                 Traceback (most recent call last)
TypeError: can only concatenate str (not "dict") to str

